fairnessを考慮したデータ表現を得るencoderをneural netを使って構築する

In [1]:
import os
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from matplotlib import pyplot as plt
%matplotlib inline

cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')
    
num_epochs = 20
batch_size = 128
learning_rate = 0.001
FEATURES_NUM=81
A_x=1
A_y=1
A_z=1
out_dir = './result/fair_autoencoder'

ModuleNotFoundError: No module named 'torch'

In [ ]:
# load data

In [ ]:
# create dataloader

In [ ]:
class fair_autoencoder(nn.Module):
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(FEATURES_NUM, 40),
            nn.ReLU(True),
            nn.Linear(40, 20),
            nn.ReLU(True),
            nn.Linear(20, 10),
            nn.ReLU(True),
            nn.Linear(10, 5),
            nn.Softmax())
        
        self.decoder = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(True),
            nn.Linear(10, 20),
            nn.ReLU(True),
            nn.Linear(20, 40),
            nn.ReLU(True),
            nn.Linear(40, FEATURES_NUM),
            nn.Tanh()
        )
        self.l1 = nn.Linear(5, 2) 

    def forward(self, x):
        z = self.encoder(x)
        xhat = self.decoder(z)
        y=self.l1(z)
        return y,xhat,z

model = fair_autoencoder()
if cuda:
    model.cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             weight_decay=1e-5)

In [ ]:
def my_loss_func(y,labels,x,xhat,z):
    L_x=nn.MSELoss(x,xhat,size_average=False)# 再構成誤差
    L_y=nn.CrossEntropyLoss(y,labels,size_average=False) # 負の対数尤度
    L_z=compute_Lz(z,x) # 公平性制約
    return A_x*L_x+A_y*L_y+A_z*L_z

def compute_Lz(z,x):
    Z_0=[]
    Z_1=[]
    for i in range(x):
        if x[i][FEATURES_NUM-1]==0:
            Z_0.append(z[i])
        else:
            Z_1.append(z[i])
    M_0=torch.mean(Z_0)
    M_1=torch.mean(Z_1)
    Lz=nn.L1Loss(M_1,M_0,size_average=False)
    return Lz

In [ ]:
def train():
    model.train()
    train_loss = 0
    for batch_idx, (data, label) in enumerate(train_loader):
        if cuda:
            data = Variable(data.cuda())
        else:
            data = Variable(data)
        optimizer.zero_grad()
        y,xhat,z= model(data)
        loss =my_loss_func(y,label, data, xhat, z)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    # loss_function() は平均ではなく全サンプルの合計lossを返すのでサンプル数で割る
    train_loss /= len(train_loader.dataset)

    return train_loss    
    

def test():
    model.eval()
    test_loss = 0
    for batch_idx, (data, label) in enumerate(test_loader):
        if cuda:
            data = Variable(data.cuda())
        else:
            data = Variable(data)
        y,xhat,z= model(data)
        loss =my_loss_func(y,label, data, xhat, z)
        test_loss += loss.item()
        
    test_loss /= len(test_loader.dataset)

    return test_loss



In [ ]:
loss_list = []
test_loss_list = []
for epoch in range(num_epochs):
    loss = train()
    test_loss = test()

    print('epoch [{}/{}], loss: {:.4f} test_loss: {:.4f}'.format(
        epoch + 1,
        num_epochs,
        loss,
        test_loss))

    # logging
    loss_list.append(loss)
    test_loss_list.append(test_loss)

# save the training model
np.save('{}/loss_list.npy'.format(out_dir), np.array(loss_list))
np.save('{}/test_loss_list.npy'.format(out_dir), np.array(test_loss_list))
torch.save(model.state_dict(), '{}/vae.pth'.format(out_dir))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
loss_list = np.load('{}/loss_list.npy'.format(out_dir))
plt.plot(loss_list)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()

In [ ]:
test_loss_list = np.load('{}/test_loss_list.npy'.format(out_dir))
plt.plot(test_loss_list)
plt.xlabel('epoch')
plt.ylabel('test loss')
plt.grid()

In [ ]:
# 中間表現の可視化　2次元の時のみ動作する

In [ ]:
model.load_state_dict(torch.load('{}/vae.pth'.format(out_dir),
                                 map_location=lambda storage,
                                 loc: storage))
data, labels = iter(test_loader).next()
data = data.view(test_batch, -1)

# 入力データを中間表現にencode
z = model.encode(Variable(images))
z.numpy()
print(z.shape)

In [ ]:
import pylab
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.scatter(z[:, 0], z[:, 1], marker='.', c=labels.numpy(), cmap=pylab.cm.jet)
plt.colorbar()
plt.grid()